In this notebook, a dataset is prepared using 2016-2022 public use NCHS U.S. mortality data (https://www.cdc.gov/nchs/nvss/mortality_public_use_data.htm). Variables of interest include age group, sex, race/ethnicity, education, place of death, manner of death (e.g., natural, suicide, homicide), and recoded proximate causes of death. 

A complex dash app is also included below, intended for use by data scientists exploring NCHS mortality data and to demonstrate how such apps can assist with exploratory data analysis. The app can be run within this notebook, but is also deployed at https://dataforpublichealth.digital/viz/nchs_2022/. 

The .csv files used in this process were created following steps outlined in accompanying notebooks at https://github.com/drjfish/nchs-mc-mortality-data-preprocessor and https://github.com/drjfish/nchs-mc-mortality-recoding-with-2022-sunburst.

In [ ]:
# Load libraries
import pandas as pd

# Load desired columns as dataframes from CSV files
cols = [
    'age_recode_27', 'sex', 'race_recode_40', 'education_2003', 'place_of_death',
    'manner_of_death', '39_cause_recode', '113_cause_recode', '358_cause_recode'
       ]

# Load CSV files using common columns
mort_2018 = pd.read_csv('mort2018_cleaned.csv', usecols=cols)
mort_2019 = pd.read_csv('mort2019_cleaned.csv', usecols=cols)
mort_2020 = pd.read_csv('mort2020_cleaned.csv', usecols=cols)
mort_2021 = pd.read_csv('mort2021_cleaned.csv', usecols=cols)
mort_2022 = pd.read_csv('mort2022_cleaned.csv', usecols=cols)

In [17]:
# Add year column to each dataframe
mort_2018['year'] = 2018
mort_2019['year'] = 2019
mort_2020['year'] = 2020
mort_2021['year'] = 2021
mort_2022['year'] = 2022

In [21]:
# Concatenate dataframes
mort_df = pd.concat([mort_2018, mort_2019, mort_2020, mort_2021, mort_2022])

# Print shape of concatenated dataframe
print(mort_df.shape)

# Save concatenated dataframe to CSV file
mort_df.to_csv('mort_2018-2022.csv', index=False)

(15859795, 10)


In [2]:
import dash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np
import plotly.express as px

# Load the primary dataset
file_path = 'mort_2018-2022.csv'
df = pd.read_csv(file_path)

# Load recode maps
recode_maps = {
    '39_cause_recode': pd.read_csv('./recodeMaps/39recodeMap.csv'),
    '113_cause_recode': pd.read_csv('./recodeMaps/113recodeMap.csv'),
    '358_cause_recode': pd.read_csv('./recodeMaps/358recodeMap.csv')
}

race_mapping = {
    1: "White",
    2: "Black",
    3: "American Indian or Alaskan Native (AIAN)",
    4: "Asian Indian",
    5: "Chinese",
    6: "Filipino",
    7: "Japanese",
    8: "Korean",
    9: "Vietnamese",
    10: "Other or Multiple Asian",
    11: "Hawaiian",
    12: "Guamanian",
    13: "Samoan",
    14: "Other or Multiple Pacific Islander",
    15: "Black and White",
    16: "Black and AIAN",
    17: "Black and Asian",
    18: "Black and Native Hawaiian or Other Pacific Islander (NHOPI)",
    19: "AIAN and White",
    20: "AIAN and Asian",
    21: "AIAN and NHOPI",
    22: "Asian and White",
    23: "Asian and NHOPI",
    24: "NHOPI and White",
    25: "Black, AIAN and White",
    26: "Black, AIAN and Asian",
    27: "Black, AIAN and NHOPI",
    28: "Black, Asian and White",
    29: "Black, Asian and NHOPI",
    30: "Black, NHOPI and White",
    31: "AIAN, Asian and White",
    32: "AIAN, NHOPI, and White",
    32: "AIAN, Asian and NHOPI",
    33: "AIAN, NHOPI and White",
    34: "Asian, NHOPI and White",
    35: "Black, AIAN, Asian and White",
    36: "Black, AIAN, Asian and NHOPI",
    37: "Black, AIAN, NHOPI and White",
    38: "Black, Asian, NHOPI and White",
    39: "AIAN, Asian, NHOPI and White",
    40: "Black, AIAN, Asian, NHOPI and White"
}

age_recode_27_mapping = {
    1: "Under 1 month",
    2: "1 month - 11 months",
    3: "1 year old",
    4: "2 years old",
    5: "3 years old",
    6: "4 years old",
    7: "5 - 9 years old",
    8: "10 - 14 years old",
    9: "15 - 19 years old",
    10: "20 - 24 years old",
    11: "25 - 29 years old",
    12: "30 - 34 years old",
    13: "35 - 39 years old",
    14: "40 - 44 years old",
    15: "45 - 49 years old",
    16: "50 - 54 years old",
    17: "55 - 59 years old",
    18: "60 - 64 years old",
    19: "65 - 69 years old",
    20: "70 - 74 years old",
    21: "75 - 79 years old",
    22: "80 - 84 years old",
    23: "85 - 89 years old",
    24: "90 - 94 years old",
    25: "95 - 99 years old",
    26: "100 years and over",
    27: "Age unknown"
}

education_mapping = {
    1: "8th grade or less",
    2: "9 - 12th grade, no diploma",
    3: "high school graduate or GED completed",
    4: "some college credit, but no degree",
    5: "Associate degree",
    6: "Bachelor’s degree",
    7: "Master’s degree",
    8: "Doctorate or professional degree",
    9: "Unknown"
}

manner_of_death_mapping = {
        1: "Accident",
        2: "Suicide",
        3: "Homicide",
        4: "Pending investigation",
        5: "Could not determine",
        6: "Self-Inflicted",
        7: "Natural"
}

place_of_death_mapping = {
        1: "Inpatient (Hospital, clinic or Medical Center)",
        2: "Outpatient (Hospital, clinic or Medical Center)",
        3: "Dead on Arrival at Hospital or Medical Facility",
        4: "Decedent’s home",
        5: "Hospice facility",
        6: "Nursing home/long term care",
        7: "Other",
        9: "Place of death unknown"
}

# Apply race and education mappings to df
df['race_description'] = df['race_recode_40'].map(race_mapping)
df['education_description'] = df['education_2003'].map(education_mapping)

# Initialize the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# App layout
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H1("2022 NCHS Mortality Data"), width=12)
    ], className="mb-4"),
    
    # Global Filters
    dbc.Row([
        dbc.Col([
            html.Label("Select Cause Recode Schema:"),
            dcc.Dropdown(
                id='recode-schema-dropdown',
                options=[
                    {'label': '39 Cause Recode', 'value': '39_cause_recode'},
                    {'label': '113 Cause Recode', 'value': '113_cause_recode'},
                    {'label': '358 Cause Recode', 'value': '358_cause_recode'}
                ],
                value='39_cause_recode',
                clearable=False
            )
        ], width=3),
        dbc.Col([
            html.Label("Select Cause Recode Value:"),
            dcc.Dropdown(
                id='recode-value-dropdown',
                clearable=False
            )
        ], width=7),
        dbc.Col([
            html.Label("Filter by Sex:"),
            dcc.Checklist(
                id='sex-filter',
                options=[{'label': 'Male', 'value': 'M'}, {'label': 'Female', 'value': 'F'}],
                value=['M', 'F'],
                inline=True,
                style={'display': 'flex', 'gap': '10px'}  # Adds space between options
            )
        ], width=2)
    ], className="mb-4"),
    
    # Top Row Visualizations
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader("Select Variable of Interest for Line Graph"),
                dbc.CardBody([
                    html.Label("X-Axis Variable:"),
                    dcc.Dropdown(
                        id='top-left-x-variable-dropdown',
                        options=[{'label': 'Year', 'value': 'year'}],
                        value='year',
                        clearable=False
                    ),
                    html.Label("Y-Axis Variable:"),
                    dcc.Dropdown(
                        id='top-left-y-variable-dropdown',
                        options=[
                            {'label': 'Count', 'value': 'count'},
                            {'label': 'Percent of Total', 'value': 'percent'}
                        ],
                        value='count',
                        clearable=False
                    ),
                    dcc.Graph(id='line-chart')
                ])
            ])
        ], width=6),
        dbc.Col([
    dbc.Card([
        dbc.CardHeader("Select Variables of Interest for Scatter Plot"),
        dbc.CardBody([
            # Full-Row Year Selector
            dbc.Row([
                dbc.Col([
                    html.Label("Select Year:"),
                    dcc.Dropdown(
                        id='year-dropdown',
                        options=[{'label': year, 'value': year} for year in sorted(df['year'].unique())],
                        value=df['year'].max(),  # Default to the latest year
                        clearable=False
                    )
                ], width=12)
            ]),
            
            # X-Axis and Y-Axis Variable Selectors in One Row
            dbc.Row([
                dbc.Col([
                    html.Label("X-Axis Variable:"),
                    dcc.Dropdown(
                        id='scatter-x-dropdown',
                        options=[{'label': col, 'value': col} for col in df.columns],
                        value='race_recode_40',
                        clearable=False
                    )
                ], width=6),
                dbc.Col([
                    html.Label("Y-Axis Variable:"),
                    dcc.Dropdown(
                        id='scatter-y-dropdown',
                        options=[{'label': col, 'value': col} for col in df.columns],
                        value='education_2003',
                        clearable=False
                    )
                ], width=6)
            ], className="mb-3"),  # Add margin-bottom for spacing
            
            # Scatter Plot Graph
            dcc.Graph(id='scatter-plot', style={'margin-top': '-0.8rem'}) # Decrease the space below the dropdowns
        ])
    ])
], width=6)
    ], className="mb-4"),
    
    # Bottom Row Visualizations
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader("Age Group Distribution"),
                dbc.CardBody([
                    html.Label("Select Year:", style={'margin-top': '0.6rem'}),
                    dcc.Dropdown(
                        id='age-year-dropdown',
                        options=[{'label': year, 'value': year} for year in df['year'].unique()] + [{'label': 'All Years', 'value': 'all'}],
                        value='all',
                        clearable=False
                    ),
                    dcc.Graph(id='age-bar-chart')
                ])
            ])
        ], width=6),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader("Horizontal Bar Chart"),
                dbc.CardBody([
            # Row for Year Selector and Radio Buttons
            dbc.Row([
                dbc.Col([
                    html.Label("Select Year:"),
                    dcc.Dropdown(
                        id='horizontal-bar-year-dropdown',
                        options=[{'label': year, 'value': year} for year in sorted(df['year'].unique())],
                        value=df['year'].max(),  # Default to the latest year
                        clearable=False
                    )
                ], width=4),  # Adjust width to make room for the radio buttons
                dbc.Col([
                    dcc.RadioItems(
                        id='horizontal-bar-option',
                        options=[
                            {'label': 'Place of Death', 'value': 'place_of_death'},
                            {'label': 'Manner of Death', 'value': 'manner_of_death'}
                        ],
                        value='place_of_death',
                        inline=True,
                        style={'display': 'flex', 'gap': '10px', 'margin-top': '0.4rem'}  # Adds space between radio options
                    )
                ], width=8)  # Adjust width to align with the dropdown
            ], className="mb-3"),  # Add margin-bottom for spacing
            
            # Graph for Horizontal Bar Chart
            dcc.Graph(id='horizontal-bar-chart')
        ])
    ])
], width=6)
    ])
], fluid=True)


@app.callback(
    [Output('recode-value-dropdown', 'options'),
     Output('recode-value-dropdown', 'value')],
    [Input('recode-schema-dropdown', 'value')]
)
def update_recode_values(selected_schema):
    # Get the corresponding recode map
    recode_map = recode_maps[selected_schema]

    # Filter recode map to include only keys present in the DataFrame
    available_keys = df[selected_schema].unique()  # Get unique keys in the current DataFrame
    filtered_recode_map = recode_map[recode_map['key'].isin(available_keys)]

    # Define the maximum length for value strings (e.g., 50 characters)
    MAX_LENGTH = 60

    # Convert to dropdown options with truncated labels
    options = [
        {
            'label': f"{row['value'][:MAX_LENGTH]}..." if isinstance(row['value'], str) and len(row['value']) > MAX_LENGTH else row['value'],
            'value': row['key']
        }
        for _, row in filtered_recode_map.iterrows()
    ]
    
    # Set the default value to the first key
    default_value = options[0]['value'] if options else None

    return options, default_value



@app.callback(
    Output('line-chart', 'figure'),
    [Input('recode-schema-dropdown', 'value'),
     Input('recode-value-dropdown', 'value'),
     Input('sex-filter', 'value'),
     Input('top-left-x-variable-dropdown', 'value'),
     Input('top-left-y-variable-dropdown', 'value')]
)
def update_line_chart(schema, value, sexes, x_variable, y_variable):
    filtered_df = df[(df[schema] == value) & (df['sex'].isin(sexes)) & (df['year'].between(2018, 2022))]
    grouped = filtered_df.groupby(['year', 'sex']).size().reset_index(name='count')
    
    if y_variable == 'percent':
        total_counts = grouped.groupby('year')['count'].transform('sum')
        grouped['percent'] = (grouped['count'] / total_counts) * 100
    
    fig = px.line(
        grouped,
        x=x_variable,
        y=y_variable,
        color='sex',
        labels={'year': 'Year', y_variable: y_variable.title()},
        color_discrete_map={'M': 'blue', 'F': 'orange'}
    )
    return fig


# Callback to populate the dropdowns dynamically and exclude specific columns
@app.callback(
    [Output('scatter-x-dropdown', 'options'),
     Output('scatter-y-dropdown', 'options')],
    Input('recode-schema-dropdown', 'value')  # Trigger dropdown population on schema change
)
def populate_dropdown_options(schema):
    # Exclude specific columns
    excluded_columns = {'sex', 'race_description', 'education_description'}
    valid_columns = [col for col in df.columns if col not in excluded_columns]

    # Generate dropdown options
    options = [{'label': col.replace('_', ' ').title(), 'value': col} for col in valid_columns]

    return options, options


# Callback to update the scatter plot
@app.callback(
    Output('scatter-plot', 'figure'),
    [Input('scatter-x-dropdown', 'value'),
     Input('scatter-y-dropdown', 'value'),
     Input('sex-filter', 'value'),
     Input('recode-schema-dropdown', 'value'),
     Input('recode-value-dropdown', 'value'),
     Input('year-dropdown', 'value')]  # Add year filter as input
)
def update_scatter_plot(x, y, sexes, schema, value, selected_year):
    # Filter the data by schema, recode value, sex, and year
    filtered_df = df[
        (df[schema] == value) &
        (df['sex'].isin(sexes)) &
        (df['year'] == selected_year)  # Filter by year
    ].copy()

    # Add jitter for plotting purposes
    jitter_strength = 0.2
    filtered_df['x_jittered'] = filtered_df[x] + np.random.uniform(-jitter_strength, jitter_strength, len(filtered_df))
    filtered_df['y_jittered'] = filtered_df[y] + np.random.uniform(-jitter_strength, jitter_strength, len(filtered_df))

    # Add hover data columns
    filtered_df['race_recode_40_raw'] = filtered_df['race_recode_40'].round().astype(int)
    filtered_df['education_2003_raw'] = filtered_df['education_2003'].round().astype(int)
    filtered_df['education_description'] = filtered_df['education_2003'].map(education_mapping)
    filtered_df['manner_of_death_desc'] = filtered_df['manner_of_death'].map(manner_of_death_mapping)
    filtered_df['place_of_death_desc'] = filtered_df['place_of_death'].map(place_of_death_mapping)
    filtered_df['age_recode_27_desc'] = filtered_df['age_recode_27'].map(age_recode_27_mapping)

    # Determine hover data dynamically
    hover_data = {
        'sex': True,  # Always show sex in hover data
        'x_jittered': False,  # Never show x_jittered
        'y_jittered': False   # Never show y_jittered
    }
    if x == 'race_recode_40' or y == 'race_recode_40':
        hover_data['race_recode_40_raw'] = True
        hover_data['race_description'] = True
    if x == 'education_2003' or y == 'education_2003':
        hover_data['education_2003_raw'] = True
        hover_data['education_description'] = True
    if x == 'manner_of_death' or y == 'manner_of_death':
        hover_data['manner_of_death'] = True
        hover_data['manner_of_death_desc'] = True
    if x == 'place_of_death' or y == 'place_of_death':
        hover_data['place_of_death'] = True
        hover_data['place_of_death_desc'] = True
    if x == 'age_recode_27' or y == 'age_recode_27':
        hover_data['age_recode_27'] = True
        hover_data['age_recode_27_desc'] = True

    # Create scatter plot
    fig = px.scatter(
        filtered_df,
        x='x_jittered',
        y='y_jittered',
        color='sex',
        color_discrete_map={'F': 'orange', 'M': 'blue'},
        labels={x: x.replace('_', ' ').title(), y: y.replace('_', ' ').title()},
        hover_data=hover_data,
    )

    # Add some transparency to make it easier to see overlapping points
    fig.update_traces(
        marker=dict(opacity=0.6)  # Set opacity to make points partially transparent
    )

    # Update axes titles dynamically
    fig.update_layout(
        xaxis_title=x.replace('_', ' ').title(),
        yaxis_title=y.replace('_', ' ').title()
    )

    return fig



@app.callback(
    Output('age-bar-chart', 'figure'),
    [Input('age-year-dropdown', 'value'),
     Input('sex-filter', 'value'),
     Input('recode-schema-dropdown', 'value'),
     Input('recode-value-dropdown', 'value')]
)
def update_age_bar_chart(selected_year, sexes, schema, value):
    # Filter the data
    filtered_df = df[(df[schema] == value) & (df['sex'].isin(sexes))].copy()
    
    # Filter by selected year
    if selected_year != 'all':
        filtered_df = filtered_df[filtered_df['year'] == selected_year]
    
    # Add age group descriptions
    filtered_df['Age Group'] = filtered_df['age_recode_27'].map(age_recode_27_mapping)
    
    # Group by age and sex
    grouped = filtered_df.groupby(['age_recode_27', 'sex', 'Age Group']).size().reset_index(name='count')
    
    # Create the bar chart
    fig = px.bar(
        grouped,
        x='age_recode_27',
        y='count',
        color='sex',
        labels={'age_recode_27': 'age_group_code', 'count': 'Count'},
        color_discrete_map={'M': 'blue', 'F': 'orange'},
        hover_data={'Age Group': True}  # Add age group description to hover
    )
    
    return fig



@app.callback(
    Output('horizontal-bar-chart', 'figure'),
    [Input('horizontal-bar-year-dropdown', 'value'),
     Input('horizontal-bar-option', 'value'),
     Input('sex-filter', 'value'),
     Input('recode-schema-dropdown', 'value'),
     Input('recode-value-dropdown', 'value')]
)
def update_horizontal_bar_chart(selected_year, option, sexes, schema, value):
    # Filter the data by schema, recode value, sex, and year
    filtered_df = df[
        (df[schema] == value) &
        (df['sex'].isin(sexes)) &
        (df['year'] == selected_year)  # Filter by selected year
    ].copy()
    
    # Add description columns
    filtered_df['manner_of_death_desc'] = filtered_df['manner_of_death'].map(manner_of_death_mapping)
    filtered_df['place_of_death_desc'] = filtered_df['place_of_death'].map(place_of_death_mapping)

    # Group by the selected option and sex, then count occurrences
    counts = filtered_df.groupby([option, 'sex']).size().reset_index(name='count')

    # Merge descriptions back into the counts DataFrame
    if option == 'manner_of_death':
        counts = counts.merge(
            filtered_df[['manner_of_death', 'manner_of_death_desc']].drop_duplicates(),
            left_on='manner_of_death',
            right_on='manner_of_death',
            how='left'
        )
    elif option == 'place_of_death':
        counts = counts.merge(
            filtered_df[['place_of_death', 'place_of_death_desc']].drop_duplicates(),
            left_on='place_of_death',
            right_on='place_of_death',
            how='left'
        )
    
    # Add hover data dynamically based on the selected option
    hover_data = {}
    if option == 'manner_of_death':
        hover_data['manner_of_death_desc'] = True
    elif option == 'place_of_death':
        hover_data['place_of_death_desc'] = True

    # Create the horizontal bar chart
    fig = px.bar(
        counts,
        x='count',
        y=option,
        color='sex',  # Color by sex
        orientation='h',
        labels={
            option: option.replace('_', ' ').title(),
            'count': 'Count',
            'sex': 'Sex'
        },
        title=f'{option.replace("_", " ").title()} Distribution for {selected_year}',
        hover_data=hover_data,
        color_discrete_map={'F': 'orange', 'M': 'blue'}  # Apply consistent colors
    )
    
    return fig


# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
